In [1]:
print("Welcome")

Welcome


In [5]:
import spacy
import pickle
import random
import json

In [15]:
import os

os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai\\research'

In [3]:
os.chdir('c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai')

In [62]:
os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai\\research'

In [16]:
file_path = '..\\data\\dataset.json'

In [22]:
with open(file_path, 'r') as file:
    data = json.load(file)

In [23]:
len(data)

1014

In [24]:
train_data = data[:800]

In [25]:
len(train_data)

800

In [49]:
test_data = data[800:]

In [51]:
len(test_data)

214

In [94]:
text_sample

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [95]:
def convert_json_to_spacy_format(json_data):
    """
    Convert JSON resume data into spaCy NER training format.
    
    Args:
        json_data (list): List of dictionaries containing resume data in the provided JSON format.
        
    Returns:
        list: List of tuples in (text, annotations) format suitable for spaCy NER training.
    """
    training_data = []
    
    for entry in json_data:
        if not isinstance(entry, list) or len(entry) != 2:
            continue
            
        text = entry[0]
        entities = entry[1].get("entities", [])
        
        # Convert entity format from [start, end, label] to (start, end, label)
        formatted_entities = []
        for entity in entities:
            if len(entity) == 3:
                formatted_entities.append(tuple(entity))
        
        training_data.append((text, {"entities": formatted_entities}))
    
    return training_data

In [96]:
convert_data = convert_json_to_spacy_format(train_data)

In [97]:
len(convert_data)

1014

In [98]:
from spacy.training import Example
from spacy.util import minibatch, compounding
from pathlib import Path

In [99]:
nlp = spacy.blank('en')

In [227]:
len(testing_data)

44

In [28]:
# print(testing_data[0][0])

In [ ]:
# def train_ner_model(training_data, model_name="resume_ner", output_dir=None, n_iter=100):
#     """
#     Train a spaCy NER model with the provided training data.
    
#     Args:
#         training_data (list): List of training examples in (text, annotations) format
#         model_name (str): Name for the new model
#         output_dir (str): Directory to save the trained model (None to not save)
#         n_iter (int): Number of training iterations
        
#     Returns:
#         spacy.Language: The trained NER model
#     """
#     # Load a blank English model or existing model
#     try:
#         nlp = spacy.load(model_name)
#         print(f"Loaded existing model '{model_name}'")
#     except:
#         nlp = spacy.blank("en")
#         print(f"Created blank 'en' model")
    
#     # Add NER pipeline if it doesn't exist
#     if "ner" not in nlp.pipe_names:
#         ner = nlp.add_pipe("ner")
#     else:
#         ner = nlp.get_pipe("ner")
    
#     # Add entity labels to the NER component
#     for _, annotations in training_data:
#         for ent in annotations.get("entities", []):
#             ner.add_label(ent[2])
    
#     # Disable other pipelines during training
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
#     with nlp.disable_pipes(*other_pipes):
#         # Initialize the model with random weights
#         nlp.initialize(lambda: [Example.from_dict(nlp.make_doc(text), annotations) 
#                       for text, annotations in training_data])
        
#         # Training loop
#         print("Beginning training...")
#         for itn in range(n_iter):
#             random.shuffle(training_data)
#             losses = {}
            
#             # Batch the examples using spaCy's minibatch
#             batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
#             for batch in batches:
#                 examples = []
#                 for text, annotations in batch:
#                     doc = nlp.make_doc(text)
#                     example = Example.from_dict(doc, annotations)
#                     examples.append(example)
                
#                 nlp.update(examples, drop=0.5, losses=losses)
            
#             print(f"Iteration {itn + 1}/{n_iter}, Loss: {losses['ner']:.4f}")
    
#     # Save the model if output directory is specified
#     if output_dir is not None:
#         output_dir = Path(output_dir)
#         if not output_dir.exists():
#             output_dir.mkdir(parents=True)
#         nlp.to_disk(output_dir)
#         print(f"Model saved to {output_dir}")
    
#     return nlp

In [41]:
import random
from pathlib import Path
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import warnings

def train_ner_model(
    training_data,
    base_model="en_core_web_sm",
    output_dir=None,
    n_iter=30,
    dropout=0.4,
    batch_sizes=(4.0, 32.0, 1.001),
    add_entity_ruler=True,
    patterns=None
):
    """Train a spaCy NER model with robust error handling."""
    
    # 1. Load model with fallback
    try:
        if base_model == "blank":
            nlp = spacy.blank("en")
            print("✓ Created blank English model")
        else:
            try:
                nlp = spacy.load(base_model)
                print(f"✓ Loaded model: {base_model}")
            except OSError:
                print(f"⚠ {base_model} not found, using en_core_web_sm")
                nlp = spacy.load("en_core_web_sm")
    except Exception as e:
        raise RuntimeError(f"❌ Model loading failed: {str(e)}") from e

    # 2. Add pipeline components
    if add_entity_ruler and "entity_ruler" not in nlp.pipe_names:
        ruler = nlp.add_pipe("entity_ruler", before="ner")
        if patterns:
            ruler.add_patterns(patterns)
        print(f"✓ Added EntityRuler with {len(patterns or [])} patterns")

    # 3. Set up NER
    ner = nlp.add_pipe("ner") if "ner" not in nlp.pipe_names else nlp.get_pipe("ner")
    
    # 4. Add labels
    labels = {ent[2] for _, ann in training_data for ent in ann.get("entities", [])}
    for label in labels:
        ner.add_label(label)
    print(f"✓ Training with {len(labels)} labels: {', '.join(labels)}")

    # 5. Initialize with suppressed warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            nlp.initialize()
        except Exception as e:
            print(f"⚠ Initialization note: {str(e)}")
            print("Proceeding with training regardless")

    # 6. Train
    print("\n🏋 Starting training...")
    disabled_pipes = [p for p in nlp.pipe_names if p not in ("ner", "entity_ruler")]
    with nlp.disable_pipes(*disabled_pipes):
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            
            batches = minibatch(training_data, size=compounding(*batch_sizes))
            for batch in batches:
                examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in batch]
                nlp.update(examples, drop=dropout, losses=losses)
            
            print(f"⟳ Iteration {itn+1}/{n_iter} | Loss: {losses.get('ner', 0):.4f}")

    # 7. Save
    if output_dir:
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        nlp.to_disk(output_path)
        print(f"\n💾 Model saved to: {output_path.resolve()}")

    return nlp

In [33]:
def filter_overlapping_entities(training_data):
    """
    Filter out overlapping entities from training data
    """
    cleaned_data = []
    
    for text, annotations in training_data:
        entities = annotations.get("entities", [])
        
        # Sort entities by start position
        entities.sort(key=lambda x: x[0])
        
        filtered_entities = []
        prev_end = -1
        
        for ent in entities:
            start, end, label = ent
            if start >= prev_end:
                filtered_entities.append(ent)
                prev_end = end
        
        cleaned_data.append((text, {"entities": filtered_entities}))
    
    return cleaned_data

In [38]:
convert_data = filter_overlapping_entities(convert_data)

In [42]:
patterns = [
    {"label": "DEGREE", "pattern": [{"LOWER": {"in": ["bsc", "b.tech", "mba"]}}]},
    {"label": "SKILL", "pattern": [{"LOWER": "python"}, {"LOWER": "tensorflow"}]}
]

# Train the model
model = train_ner_model(
    train_data,
    base_model="en_core_web_lg",
    output_dir="./resume_ner",
    n_iter=50,
    patterns=patterns
)

⚠ en_core_web_lg not found, using en_core_web_sm
✓ Added EntityRuler with 2 patterns
✓ Training with 18 labels: EMAIL ADDRESS, DESIGNATION, COLLEGE NAME, DEGREE, CERTIFICATION, YEAR OF GRADUATION, LINKEDIN LINK, AWARDS, NAME, Unlabelled, COMPANIES WORKED AT, LOCATION, CONTACT, SKILLS, YEARS OF EXPERIENCE, UNIVERSITY, WORKED AS, LANGUAGE
⚠ Initialization note: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.
Proceeding with training regardless

🏋 Starting training...


c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "   
Contact
www.linkedin.com/in/akhil-shah04(Linke..." with entities "[[12, 44, 'LINKEDIN LINK'], [66, 70, 'SKILLS'], [7...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "   
Contact
www.linkedin.com/in/
deepakpandit16  (..." with entities "[[12, 47, 'LINKEDIN LINK'], [60, 73, 'NAME'], [138...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\arshi\Downloads\Desktop\Bro-P

⟳ Iteration 1/50 | Loss: 36396.2500
⟳ Iteration 2/50 | Loss: 28034.5391
⟳ Iteration 3/50 | Loss: 24916.1816
⟳ Iteration 4/50 | Loss: 20241.9824
⟳ Iteration 5/50 | Loss: 17552.5938
⟳ Iteration 6/50 | Loss: 16465.2305
⟳ Iteration 7/50 | Loss: 16164.0596
⟳ Iteration 8/50 | Loss: 14647.7686
⟳ Iteration 9/50 | Loss: 13779.7578
⟳ Iteration 10/50 | Loss: 13392.1104
⟳ Iteration 11/50 | Loss: 12895.5117
⟳ Iteration 12/50 | Loss: 12157.5625
⟳ Iteration 13/50 | Loss: 11892.2441
⟳ Iteration 14/50 | Loss: 11249.0225
⟳ Iteration 15/50 | Loss: 11155.6719
⟳ Iteration 16/50 | Loss: 10614.6006
⟳ Iteration 17/50 | Loss: 10387.0479
⟳ Iteration 18/50 | Loss: 10444.7646
⟳ Iteration 19/50 | Loss: 9939.2168
⟳ Iteration 20/50 | Loss: 10265.3838
⟳ Iteration 21/50 | Loss: 9468.2432
⟳ Iteration 22/50 | Loss: 9216.0137
⟳ Iteration 23/50 | Loss: 9818.1602
⟳ Iteration 24/50 | Loss: 9266.4648
⟳ Iteration 25/50 | Loss: 8705.7402
⟳ Iteration 26/50 | Loss: 8673.6719
⟳ Iteration 27/50 | Loss: 8787.0605
⟳ Iteration 28/50 

In [43]:
test_text = "'Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN   SKILLS  APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)  LINKS  https://www.linkedin.com/in/govardhana-k-61024944/  ADDITIONAL INFORMATION  Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/"
doc = model(test_text)
print("\nTest prediction:")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")
   
# skills = [ent.text for ent in doc.ents if ent.label_ == 'Skills']


Test prediction:
Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ (LOCATION)
Senior Consultant (WORKED AS)
Associate Consultant (WORKED AS)
Adithya Institute of Technology -  Tamil Nadu (COLLEGE NAME)
Less than 1 year (YEARS OF EXPERIENCE)
3 years (YEARS OF EXPERIENCE)
5 years (YEARS OF EXPERIENCE)
5 years (YEARS OF EXPERIENCE)
3 years (YEARS OF EXPERIENCE)
JavaScript (SKILLS)
XML (SKILLS)
HTML (SKILLS)


In [44]:
# train_data[0]

In [46]:
skills

['APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)',
 'Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/']

In [45]:
from spacy.scorer import Scorer
from spacy.training import Example

In [46]:
os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai\\research'

In [64]:
nlp = spacy.load("..\\models\\resume_ner")

In [48]:
def evaluate_ner(model, eval_data):
    scorer = Scorer()
    examples = []
    
    for text,annotations in eval_data: 
        doc = model.make_doc(text)
        example = Example.from_dict(doc, annotations)
        example.predicted = model(text)
        examples.append(example)
    scores = scorer.score(examples)
    return {
        'precision': scores['ents_p'],
        'recall': scores['ents_r'],
        'f1_score': scores['ents_f'],
        'per_type': scores['ents_per_type']
    }

In [52]:
results = evaluate_ner(nlp, test_data )

c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "   
Contact
www.linkedin.com/in/som-
kumar-48aa696..." with entities "[[12, 51, 'LINKEDIN LINK'], [63, 72, 'NAME'], [73,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "   
Contact
www.linkedin.com/in/sharad52
(LinkedIn..." with entities "[[12, 40, 'LINKEDIN LINK'], [63, 73, 'SKILLS'], [7...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\arshi\Downloads\Desktop\Bro-P

In [53]:
results

{'precision': 0.7905616224648986,
 'recall': 0.7800654223590533,
 'f1_score': 0.7852784503631961,
 'per_type': {'LINKEDIN LINK': {'p': 0.9953051643192489,
   'r': 0.9464285714285714,
   'f': 0.9702517162471396},
  'SKILLS': {'p': 0.7273809523809524,
   'r': 0.6729074889867841,
   'f': 0.6990846681922197},
  'NAME': {'p': 0.8341968911917098,
   'r': 0.7892156862745098,
   'f': 0.8110831234256927},
  'LOCATION': {'p': 0.8241206030150754,
   'r': 0.841025641025641,
   'f': 0.83248730964467},
  'COMPANIES WORKED AT': {'p': 0.8421052631578947,
   'r': 0.7851458885941645,
   'f': 0.8126286890871653},
  'WORKED AS': {'p': 0.85, 'r': 0.8617511520737328, 'f': 0.8558352402745996},
  'YEARS OF EXPERIENCE': {'p': 0.9052511415525114,
   'r': 0.9790123456790123,
   'f': 0.9406880189798339},
  'COLLEGE NAME': {'p': 0.4626865671641791,
   'r': 0.6526315789473685,
   'f': 0.5414847161572052},
  'DEGREE': {'p': 0.7018181818181818,
   'r': 0.7228464419475655,
   'f': 0.7121771217712177},
  'CERTIFICATION

## Second way to create model using span

In [53]:
def convert_to_spans_format(training_data, nlp):
    """
    Convert entities to spans format with validation for:
    - Invalid offsets (start > end)
    - Offsets exceeding text length
    - Proper token alignment
    """
    spans_data = []
    error_count = 0
    
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        spans = []
        
        for ent in annotations.get("entities", []):
            if len(ent) != 3:
                continue
                
            start, end, label = ent
            
            # Validate offsets
            if start >= end:
                error_count += 1
                continue
                
            if end > len(text):
                error_count += 1
                continue
                
            span = doc.char_span(start, end, label=label)
            if span is not None:
                spans.append(span)
            else:
                error_count += 1
        
        spans_data.append((text, {"spans": spans}))
    
    if error_count > 0:
        print(f"Warning: Skipped {error_count} invalid entity spans")
    
    return spans_data

In [56]:
nlp1 = spacy.blank("en")

In [54]:
convert_data_span = filter_overlapping_entities(train_data)

In [57]:
train_data_span = convert_to_spans_format(convert_data,nlp1)

In [59]:
def train_ner_model_span(training_data, model_name="resume_ner", output_dir=None, n_iter=100):
    """Train with error handling for overlapping entities"""
    # Load or create model
    try:
        nlp = spacy.load(model_name)
        print(f"Loaded existing model '{model_name}'")
    except:
        nlp = spacy.blank("en")
        print(f"Created blank 'en' model")
    
    # Add NER pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    # Add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities", []):
            ner.add_label(ent[2])
    
    # Training with error handling
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.initialize()
        
        print("Beginning training...")
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                examples = []
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    try:
                        example = Example.from_dict(doc, annotations)
                        examples.append(example)
                    except ValueError as e:
                        print(f"Skipping problematic example: {str(e)}")
                        continue
                
                if examples:  # Only update if we have valid examples
                    nlp.update(examples, drop=0.5, losses=losses, sgd=optimizer)
            
            print(f"Iteration {itn + 1}/{n_iter}, Loss: {losses.get('ner', 0):.4f}")
    
    if output_dir:
        output_dir = Path(output_dir)
        output_dir.mkdir(exist_ok=True)
        nlp.to_disk(output_dir)
        print(f"Model saved to {output_dir}")
    
    return nlp

In [60]:
trained_model2 = train_ner_model_span(
    train_data_span,
    model_name="resume_ner2",
    output_dir="./resume_ner_model2",
    n_iter=50
)

Created blank 'en' model
Beginning training...
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be optionally extended with a label and a KB ID.
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be optionally extended with a label and a KB ID.
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be optionally extended with a label and a KB ID.
Skipping problematic example: [E879] Unexpected type for 'spans' data. Provide a dictionary mapping keys to a list of spans, with each span represented by a tuple (start_char, end_char). The tuple can be

In [87]:
test_text = "Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-download&ikw=download-top&co=IN   SKILLS  APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)  LINKS  https://www.linkedin.com/in/govardhana-k-61024944/  ADDITIONAL INFORMATION  Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/"
doc = trained_model(test_text)
print("\nTest prediction:")
# for ent in doc.ents:
#     print(f"{ent.text} ({ent.label_})")
   
skills2 = [ent.text for ent in doc.ents if ent.label_ == 'Skills']


Test prediction:


In [88]:
skills2

['APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years), Algorithms (3 years)',
 'Technical Proficiency:  Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle PL-SQL programming, Sales Force with APEX. Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer, PL/SQL Developer, WinSCP, Putty Web Technologies: JavaScript, XML, HTML, Webservice  Operating Systems: Linux, Windows Version control system SVN & Git-Hub Databases: Oracle Middleware: Web logic, OC4J Product FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x  https://www.linkedin.com/in/govardhana-k-61024944/']

In [63]:
print('model creation failed')

model creation failed


In [29]:
os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai\\research'

In [30]:
import spacy

In [63]:
nlp = spacy.load("..\\models\\resume_ner")

In [55]:
from pathlib import Path
from PyPDF2 import PdfReader 

In [56]:
with open("..\\data\\Arshid.pdf",'rb') as file:
    pdf_reader = PdfReader(file)
    full_text = ""
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:  # Only add if text exists
            full_text += page_text + "\n"  # Add newline between pages
    
    # Print the extracted text
    print(full_text)

Arshid T M
DATA SCIENTIST
♂¶ap-¶arker-altKannur,Kerala /envel⌢pearshidtm2001@gmail.com ♂phone-alt+91 8078196576 < / > Leetcode /linkedin-inArshid T M
/githubArshidtm
Profile Summary
Data Science and Machine Learning enthusiast with expertise in building predictive models, deep learning,
and NLP. Skilled in data preprocessing, feature engineering, and EDA. Proficient in Python, TensorFlow, and
Scikit-Learn for model development and deployment. Passionate about leveraging AI to solve real-world problems
and drive business growth.
Skills
◦Programming Language : Python
◦Data Analysis & Visualization : Pandas, NumPy, Matplotlib, Seaborn, Tableau
◦Database Management : PostgreSQL
◦Machine Learning :
– Regression : Linear Regression, Random Forest Regressor
– Classification : KNN, Naive Bayes, SVM, Decision Trees, Random Forest Classifier
– Ensemble Learning : Bagging, XgBoost
– Clustering : K-Means, DBSCAN
– Advanced Topics : Natural Language Processing (NLP), Deep Learning
Projects
Sentimen

In [57]:
doc = nlp(full_text)
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")

Pandas (SKILLS)
NumPy (SKILLS)
Matplotlib (SKILLS)
Seaborn (SKILLS)
Sentiment Analysis GitHub (CERTIFICATION)
Mini Project (COMPANIES WORKED AT)
Data Science (COLLEGE NAME)
Master of Science in Data Science and Business Analysis (DEGREE)
Rathinam College of Arts and Science - Bharatiyar University, Coimbatore2022 - 2024 (COMPANIES WORKED AT)
CGPA: 7.9 (CERTIFICATION)
Bachelor of Science in Mathematics
Sir Syed College - Kannur University, Kannur2019 - 2022 (DEGREE)
CGPA: 7.067 (CERTIFICATION)


In [58]:
skills = [ent.text for ent in doc.ents if ent.label_ == 'SKILLS' or ent.label_ == "CERTIFICATION" or ent.label_ == "DEGREE"]

In [59]:
skills

['Pandas',
 'NumPy',
 'Matplotlib',
 'Seaborn',
 'Sentiment Analysis GitHub',
 'Master of Science in Data Science and Business Analysis',
 'CGPA: 7.9',
 'Bachelor of Science in Mathematics\nSir Syed College - Kannur University, Kannur2019 - 2022',
 'CGPA: 7.067']

In [60]:
skills = "This is my extracted data from my resume \n" + " ".join(skills)

In [61]:
print(skills)

This is my extracted data from my resume 
Pandas NumPy Matplotlib Seaborn Sentiment Analysis GitHub Master of Science in Data Science and Business Analysis CGPA: 7.9 Bachelor of Science in Mathematics
Sir Syed College - Kannur University, Kannur2019 - 2022 CGPA: 7.067


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [3]:
embeddings = HuggingFaceEmbeddings()

c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Much smaller (~80MB)
embeddings2 = HuggingFaceEmbeddings(model_name=model_name)

c:\Users\arshi\Downloads\Desktop\Bro-Project\SkillForge.ai\env\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arshi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [114]:
import os 
os.getcwd()

'c:\\Users\\arshi\\Downloads\\Desktop\\Bro-Project\\SkillForge.ai\\research'

In [115]:
vector_store = FAISS.load_local(
    folder_path="job_vector_db",  # Path to saved folder
    embeddings=embeddings,
    allow_dangerous_deserialization = True
)

In [ ]:
# system_prompt = ("""
# You are a friendly and knowledgeable AI Career Mentor. Your role is to analyze skill gaps between a user's extracted resume skills and job descriptions from LinkedIn. You will provide insightful and practical recommendations to help users improve their qualifications and bridge any skill gaps. Be honest, supportive, and solution-oriented.

# Instructions:
# 1. If the context provided is a casual greeting such as "Hi", "Hello", or "How are you?", respond briefly by introducing who you are. Avoid adding any unnecessary or unrelated information
# 2. Carefully analyze the context provided, which includes the user's extracted skills and job requirements from the vector store.
# 3 Identify missing or underdeveloped skills and suggest actionable steps to bridge these gaps. 
#    - Provide resources like courses, certifications, or projects.
#    - Recommend practical experience or networking opportunities where relevant.
# 4 Offer clear, concise, and easy-to-understand responses tailored to the user's career growth.
# 5. If there is insufficient information in the retrieved data to answer accurately, clearly state that the information is not available instead of attempting to generate a speculative response. Encourage the user to provide additional details or rephrase the query
# 6. Provide alternative options where possible and avoid making up information or giving speculative responses.
# 7. Do not treat the vector store data as a specific job description — it is intended for understanding market-relevant skills.
# Context: {context}
# """)


In [116]:
system_prompt = ("""
You are a friendly and knowledgeable AI Career Mentor. Your role is to analyze skill gaps between a user's extracted resume skills and job descriptions from LinkedIn. You will provide insightful and practical recommendations to help users improve their qualifications and bridge any skill gaps. Be honest, supportive, and solution-oriented.

Instructions:
Respond to questions within the scope of the provided vector store, which includes job descriptions and skills for technical roles ,  If a question is outside this scope, please respond with 'No relevant information found in the vector store.'"
1. If the context provided is a casual greeting such as "Hi", "Hello", or "How are you?", respond briefly by introducing who you are. Avoid adding any unnecessary or unrelated information.
2. Carefully analyze the context provided, which includes the user's extracted skills and job requirements from the vector store. Ensure you have a thorough understanding of the user's skills and the market-relevant skills retrieved from the vector store.
3. Identify missing or underdeveloped skills and suggest actionable steps to bridge these gaps. 
   - Provide resources like courses, certifications, or projects that are directly relevant to the identified skill gaps.
   - Recommend practical experience or networking opportunities where relevant and feasible.
4. Offer clear, concise, and easy-to-understand responses tailored to the user's career growth. Avoid using jargon or technical terms that may confuse the user.
5. "Only generate responses based on context retrieved from the vector store. If no relevant context is found, respond with 'I don't know' or 'No information available'. Avoid generating speculative or generic responses. Provide accurate and relevant information only when it can be retrieved from the vector store."
6. Provide alternative options where possible and avoid making up information or giving speculative responses. If you're unsure or lack sufficient information to provide an accurate answer, say so and ask for clarification or more context.
7. Do not treat the vector store data as a specific job description — it is intended for understanding market-relevant skills. Keep your responses focused on the user's skills and career growth, rather than specific job openings.
8. Always prioritize accuracy and transparency in your responses. If you're unable to provide a helpful answer, say so and encourage the user to seek additional resources or guidance.

Context: {context}
""")

In [81]:
retriever=vector_store.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [82]:
type(retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [83]:
contextualize_q_system_prompt  = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "Formulate a standalone query which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return as it is."
)

In [84]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain,create_history_aware_retriever

In [85]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    
    
)

In [86]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ('human',"{input}"),
    ]
)

In [87]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)


In [88]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [89]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [90]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [91]:
user_input1 = "What are the skills required for an AI engineer"

In [92]:
response1 = conversational_rag_chain.invoke(
    {"input":user_input1 },
    config={
        "configurable": {"session_id": "abc1234"}
    },  # constructs a key "abc123" in `store`.
)

In [73]:
# store['abc1234']

In [94]:
print(response1['answer'])

Based on the context provided, the skills required for an AI/ML Developer or AI Engineer include:

**Required Skills:**

1. **AI/ML development experience**: 2+ years of professional experience in AI/ML development, with a focus on ML, DL, and Computer Vision.
2. **Large Language Models (LLM)**: Experience with LLM and Retrieval-Augmented Generation (RAG) frameworks.
3. **Python programming**: Proficiency in Python programming and hands-on experience with ML/DL libraries such as TensorFlow, PyTorch, OpenCV, etc.
4. **Machine Learning and Deep Learning**: Experience in designing, training, and optimizing machine learning and deep learning models, including computer vision tasks.
5. **Computer Vision**: Solid understanding of computer vision techniques (image recognition, object detection, segmentation).
6. **REST APIs**: Strong experience in building and deploying REST APIs using Flask, FastAPI, or Django.
7. **Problem-solving skills**: Strong problem-solving skills with the ability to 

In [95]:
user_input2 = "tell me more about point number 5"

In [96]:
response2 = conversational_rag_chain.invoke(
    {"input": user_input2},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [97]:
print(response2['answer'])

Point number 5 refers to **Computer Vision**. Computer Vision is a field of study that enables computers to interpret and understand visual information from the world. It is a crucial aspect of Artificial Intelligence (AI) and Machine Learning (ML) that allows computers to extract meaningful information from images and videos.

**Key Computer Vision Techniques:**

1. **Image Recognition**: The ability of a computer to identify and classify objects within an image.
2. **Object Detection**: The ability of a computer to locate and classify objects within an image or video.
3. **Segmentation**: The ability of a computer to partition an image into its constituent parts or objects.
4. **Image Processing**: The ability of a computer to enhance, transform, or manipulate images.

**Applications of Computer Vision:**

1. **Self-Driving Cars**: Computer vision is used to detect and respond to objects on the road, such as pedestrians, cars, and traffic lights.
2. **Facial Recognition**: Computer v

In [26]:
print(response2['chat_history'])

[HumanMessage(content='What are the skills required for an AI engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="As a Career Mentor, I've analyzed the context provided, which includes the required skills for an AI/ML Developer role. The key skills required for an AI engineer include:\n\n1. **Programming skills**: Proficiency in Python programming is essential, along with hands-on experience with ML/DL libraries such as TensorFlow, PyTorch, OpenCV, etc.\n2. **Machine Learning and Deep Learning**: Experience in designing, training, and optimizing machine learning and deep learning models, including computer vision tasks.\n3. **Large Language Models (LLM)**: Experience with LLMs and Retrieval-Augmented Generation (RAG) frameworks.\n4. **Computer Vision**: Solid understanding of computer vision techniques, including image recognition, object detection, segmentation, and video analysis.\n5. **REST APIs**: Strong experience in building and deploying REST APIs using fr

In [98]:
user_input3 = "What are class based views in django"

In [99]:
response3 = conversational_rag_chain.invoke(
    {"input": user_input3},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [101]:
print(response3['context'])

[Document(id='a4a7096d-7c79-4ffc-9684-5dd6adfbc7a4', metadata={'job_title': 'iOS Team Lead', 'company': 'Optimumbrew Technology', 'location': 'India', 'searched_city': 'Surat', 'language': 'en'}, page_content='Roles \\& Responsibilities'), Document(id='4eba5d9f-8abd-4ec9-abc4-f91bdd5b4daf', metadata={'job_title': 'AI/ML Developer', 'company': 'WebOsmotic Private Limited', 'location': 'Surat, Gujarat, India', 'searched_city': 'Surat', 'language': 'en'}, page_content='AI/ML Developer\n \nRole \\& Responsibilities: \n\n* Develop, fine\\-tune, and optimize\xa0\n Large Language Models (LLMs) \n \xa0and\xa0\n Deep Learning models \n \xa0for AI and Computer Vision applications.\n* Implement and optimize\xa0\n Retrieval\\-Augmented Generation (RAG) \n \xa0frameworks to enhance AI model performance.\n* Apply\xa0\n agentic LLM frameworks \n \xa0to build autonomous AI agents for real\\-world use cases.\n* Develop\xa0\n computer vision models \n \xa0for tasks such as\xa0\n image recognition, objec

In [102]:
print(response3['input'])

What are class based views in django


In [103]:
user_input4 = "can you tell me about a movie"

In [104]:
response4 = conversational_rag_chain.invoke(
    {"input": user_input4},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [105]:
print(response4['answer'])

I'd be happy to tell you about a movie. However, I need to know which movie you're interested in. There are thousands of movies out there, and I don't want to recommend one that you might not be interested in.

Could you please give me some more information about the type of movie you're in the mood for? For example:

* What genre are you interested in? (e.g. action, comedy, romance, horror, etc.)
* Are you looking for a classic movie or something more recent?
* Is there a particular actor or actress you'd like to see in the movie?
* Are you open to watching a foreign film or do you prefer movies in a specific language?

Let me know, and I'll do my best to recommend a movie that fits your interests!


In [106]:
user_input6 = 'tell me about jhon wick'

In [107]:
response6 = conversational_rag_chain.invoke(
    {"input": user_input6},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [109]:
print(response6['context'])

[Document(id='4eba5d9f-8abd-4ec9-abc4-f91bdd5b4daf', metadata={'job_title': 'AI/ML Developer', 'company': 'WebOsmotic Private Limited', 'location': 'Surat, Gujarat, India', 'searched_city': 'Surat', 'language': 'en'}, page_content='AI/ML Developer\n \nRole \\& Responsibilities: \n\n* Develop, fine\\-tune, and optimize\xa0\n Large Language Models (LLMs) \n \xa0and\xa0\n Deep Learning models \n \xa0for AI and Computer Vision applications.\n* Implement and optimize\xa0\n Retrieval\\-Augmented Generation (RAG) \n \xa0frameworks to enhance AI model performance.\n* Apply\xa0\n agentic LLM frameworks \n \xa0to build autonomous AI agents for real\\-world use cases.\n* Develop\xa0\n computer vision models \n \xa0for tasks such as\xa0\n image recognition, object detection, segmentation, and video analysis \n .\n* Integrate AI and Computer Vision models into production systems through\xa0\n REST APIs \n \xa0and backend frameworks (\n Flask, FastAPI, Django \n ).\n* Collaborate with cross\\-functi

In [110]:
user_input7 = "what is this how can you generate this ? this is not in the vector store so how you generate this?"

In [111]:
response7 =conversational_rag_chain.invoke(
    {"input": user_input7},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jp4qhsk5enfvzcds5rj2hy78` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99531, Requested 4050. Please try again in 51m33.978s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [37]:
print(response7['answer'])

You're right, the text I generated about the movie "Ex Machina" is not in the vector store. I generated this text using a combination of natural language processing (NLP) and machine learning algorithms.

Here's a high-level overview of how I generated this text:

1. **Knowledge Retrieval**: I have been trained on a massive corpus of text data, which includes information about various topics, including movies. When you asked me about a movie, I retrieved relevant information from my knowledge base.
2. **Text Generation**: I used a text generation algorithm to create a coherent and readable text based on the information I retrieved. This algorithm takes into account various factors such as grammar, syntax, and style to generate a text that is similar to human-written text.
3. **Language Model**: My language model is based on a transformer architecture, which is a type of neural network that is well-suited for NLP tasks. This model allows me to generate text that is context-dependent and

In [38]:
user_input5 = "i dont want to generate this type of response . so give me prompt to make changes in my system prompt"

In [39]:
response8 = conversational_rag_chain.invoke(
    {"input": user_input5},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [40]:
print(response8['answer'])

To change the way I respond to your prompts, you can modify the system prompt to include specific instructions or guidelines. Here are a few options:

1. **Add a specific topic or domain**: You can add a specific topic or domain to the system prompt to limit my responses to that area. For example:
"Provide responses related to career development and job skills, without generating examples or anecdotes."
2. **Specify a response format**: You can specify a response format, such as a list or a step-by-step guide, to help me provide more structured responses. For example:
"Provide responses in a numbered list format, with each point addressing a specific aspect of the topic."
3. **Emphasize conciseness**: You can add a prompt to emphasize conciseness and brevity in my responses. For example:
"Provide brief and to-the-point responses, avoiding unnecessary examples or explanations."
4. **Focus on technical aspects**: You can add a prompt to focus on technical aspects of a topic, avoiding non

In [41]:
user_input9 = "give me prompt like , only generate reponse when you can able to retrieve context from the vectore store .otherwise generate you dont know or something. "

In [42]:
response9 = conversational_rag_chain.invoke(
    {"input": user_input9},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [43]:
print(response9['answer'])

Here is a prompt that you can use:

"Only generate responses based on context retrieved from the vector store. If no relevant context is found, respond with 'I don't know' or 'No information available'. Avoid generating speculative or generic responses. Provide accurate and relevant information only when it can be retrieved from the vector store."

Alternatively, you can use a shorter version:

"Respond only with context from the vector store. Otherwise, say 'I don't know'."

This prompt instructs me to only generate responses when I can retrieve relevant context from the vector store, and to avoid providing speculative or generic responses. If I'm unable to find relevant context, I will respond with a message indicating that I don't know or that no information is available.


In [ ]:
"Only generate responses based on context retrieved from the vector store. If no relevant context is found, respond with 'I don't know' or 'No information available'. Avoid generating speculative or generic responses. Provide accurate and relevant information only when it can be retrieved from the vector store."

In [44]:
user_input10 = "can you act like that .like i updated the system prompt to the text you provided . so you can generate response based on the retrieved context"

In [45]:
response10 = conversational_rag_chain.invoke(
    {"input": user_input10},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [46]:
print(response10['answer'])

I will act as if the system prompt has been updated to:

"Only generate responses based on context retrieved from the vector store. If no relevant context is found, respond with 'I don't know' or 'No information available'. Avoid generating speculative or generic responses. Provide accurate and relevant information only when it can be retrieved from the vector store."

From now on, I will only generate responses based on the context retrieved from the vector store. If I don't find any relevant context, I will respond with "I don't know" or "No information available".

Please go ahead and ask your question or provide a prompt. I will respond accordingly.


In [47]:
user_input11 = "can you generate some thing that is based on the vectore store i provided"

In [48]:
response11 = conversational_rag_chain.invoke(
    {"input": user_input11},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [49]:
print(response11['answer'])

Based on the vector store you provided, I can see that it contains information about a job posting for a "Consultant, Data Science and Analytics" role at TransUnion.

Here is a response based on the retrieved context:

"The job posting for Consultant, Data Science and Analytics at TransUnion requires a strong background in data science and analytics, with experience in machine learning, statistical modeling, and data visualization. The ideal candidate will have a degree in a quantitative field, such as mathematics, statistics, or computer science, and will be proficient in programming languages such as Python, R, or SQL."

Please let me know if this response is accurate and relevant to the context retrieved from the vector store. If I'm unsure or don't have enough information, I will respond with "I don't know" or "No information available".


In [50]:
user_input12 = "what about mern stack"

In [51]:
response12 = conversational_rag_chain.invoke(
    {"input": user_input12},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [52]:
print(response12['answer'])

I don't know. The vector store does not contain any information about the MERN stack.


In [53]:
user_input13 = "what about django"

In [54]:

response13 = conversational_rag_chain.invoke(
    {"input": user_input13},
    config={
        "configurable": {"session_id": "abc1234"}
        ,"max_tokens": 1024
    },  # constructs a key "abc123" in `store`.
)

In [55]:
print(response13['answer'])

Based on the vector store, I can see that there is information about Django, specifically about Class-Based Views in Django.

The vector store contains context about how Class-Based Views work in Django, including the benefits of using them, how to define a view as a class, and how to handle different HTTP methods.

If you would like to know more about Class-Based Views in Django, I can try to provide more information based on the context retrieved from the vector store.


## Second try

In [118]:
# Load vector store with error handling
try:
    vector_store = FAISS.load_local(
        folder_path="job_vector_db",
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )
except Exception as e:
    raise ValueError(f"Failed to load vector store: {str(e)}")

In [ ]:

# system_prompt = """
# You are a career mentor AI STRICTLY LIMITED to discussing technical job skills and career advice. 

# RULES YOU MUST FOLLOW:
# 1. ONLY respond using information from the provided context documents
# 2. If no relevant context exists, respond EXACTLY: "I can only answer questions about technical career skills based on my knowledge base."
# 3. Never speculate, invent, or hallucinate information
# 4. For greetings, respond ONLY with: "Hello! I'm your technical career mentor. How can I help?"
# 5. If asked about topics outside technical careers, respond: "I specialize in technical career advice only."

# Context documents: {context}
# """

In [198]:
system_prompt = ("""
You are a friendly and knowledgeable AI Career Mentor. Your role is to analyze skill gaps between a user's extracted resume skills and job descriptions from LinkedIn. You will provide insightful and practical recommendations to help users improve their qualifications and bridge any skill gaps. Be honest, supportive, and solution-oriented.

Instructions:
Respond to questions within the scope of the provided vector store, which includes job descriptions and skills for technical roles, don't mention about the company name or anything related to company from the vector sctore ,  If a question is outside this scope, please respond with 'No relevant information found in the vector store.'"
1. If the context provided is a casual greeting such as "Hi", "Hello", or "How are you?", respond briefly by introducing who you are. Avoid adding any unnecessary or unrelated information.
2. Carefully analyze the context provided, which includes the user's extracted skills and job requirements from the vector store. Ensure you have a thorough understanding of the user's skills and the market-relevant skills retrieved from the vector store.
3. Identify missing or underdeveloped skills and suggest actionable steps to bridge these gaps. 
   - Provide resources like courses, certifications, or projects that are directly relevant to the identified skill gaps.
   - Recommend practical experience or networking opportunities where relevant and feasible.
4. Offer clear, concise, and easy-to-understand responses tailored to the user's career growth. Avoid using jargon or technical terms that may confuse the user.
5. "Only generate responses based on context retrieved from the vector store. If no relevant context is found, respond with 'I don't know' or 'No information available'. Avoid generating speculative or generic responses. Provide accurate and relevant information only when it can be retrieved from the vector store."
6. Provide alternative options where possible and avoid making up information or giving speculative responses. If you're unsure or lack sufficient information to provide an accurate answer, say so and ask for clarification or more context.
7. Do not treat the vector store data as a specific job description — it is intended for understanding market-relevant skills. Keep your responses focused on the user's skills and career growth, rather than specific job openings.
8. Always prioritize accuracy and transparency in your responses. If you're unable to provide a helpful answer, say so and encourage the user to seek additional resources or guidance.

Context: {context}
""")

In [199]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        'k': 3,
        'score_threshold': 0.5  
    }
)

In [200]:
contextualize_q_system_prompt = """
Given a chat history and the latest user question which might reference context in the chat history,
formulate a standalone query which can be understood without the chat history.
IMPORTANT: If the question is clearly outside technical career topics, return "OFFTOPIC".

Do NOT answer the question, just reformulate it if needed.
"""

In [201]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",  
    temperature=0.3, 
    max_tokens=1500
)

In [202]:
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [203]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [204]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [205]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [206]:
def context_filter(input_dict):
    if not input_dict.get("context"):
        return {"answer": "I can only answer questions about technical career skills based on my knowledge base."}
    
    # Additional check for document relevance
    # if all(doc.metadata.get('relevance_score', 0) < 0.3 for doc in input_dict["context"]):
    #     return {"answer": "I don't have enough relevant information about this specific technical career question."}
    
    return input_dict

In [207]:
from langchain_core.runnables import RunnableLambda

In [ ]:
base_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
filtered_rag_chain = base_rag_chain | RunnableLambda(context_filter)

In [209]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [210]:
conversational_rag_chain = RunnableWithMessageHistory(
    filtered_rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [211]:
def get_response(user_input, session_id="default_session"):
    response = conversational_rag_chain.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )
    return response["answer"]

In [212]:
user_input1 = "what are the skills required for data scientist"

In [213]:
session_id = "abcd123"

In [215]:
responses1 = get_response(user_input1,session_id)
print(responses1)

Based on the context provided, the skills required for a Data Scientist role include:

1. **Programming skills**: Proficiency in languages such as Python.
2. **Data analysis and management**: Ability to work with databases, including SQL.
3. **Machine learning and deep learning**: Experience with machine learning and deep learning algorithms.
4. **Cloud services**: Proficiency in cloud services such as Azure.
5. **Data visualization and communication**: Ability to collect and synthesize information, making it relevant, understandable, and actionable for key stakeholders.
6. **Analytics and problem-solving**: Strong understanding of Analytics and Visualization techniques, with ability to generate insights that improve the business.
7. **Generative AI and LangChain**: Experience with generative AI and LangChain for building and deploying advanced models.

Additionally, the following skills are mentioned as required:

* **Machine Learning**
* **Deep Learning**
* **Pytorch**
* **Cloud**
* 

In [217]:
user_input14 = "tell me more about point 4"

In [218]:
responses2 = get_response(user_input14,session_id)
print(responses2)

Point 4 refers to **Cloud services**: Proficiency in cloud services such as Azure. As a Data Scientist, having experience with cloud services is important because it allows you to:

1. **Implement and manage AI solutions**: Cloud services provide a scalable and secure environment to deploy and manage AI models.
2. **Store and process large datasets**: Cloud services offer vast storage and computing resources to handle large datasets, making it easier to analyze and process data.
3. **Collaborate with teams**: Cloud services enable seamless collaboration with cross-functional teams, including data engineers, developers, and other stakeholders.

Some key skills related to cloud services for a Data Scientist include:

1. **Azure services**: Experience with Azure services such as Azure Machine Learning, Azure Storage, and Azure Databricks.
2. **Cloud-based data platforms**: Familiarity with cloud-based data platforms such as Azure Synapse Analytics, Azure Data Factory, and Azure Cosmos DB.

In [179]:
user_input15 = "What about Mern stack?"

In [220]:
responses3 = get_response(user_input15, session_id)
print(responses3)

The MERN stack is not a primary requirement for a Data Scientist role. The skills required for a Data Scientist include:

1. **Machine learning and deep learning**
2. **Data analysis and visualization**
3. **Statistics and mathematics**
4. **Programming languages like Python and R**
5. **Data management and storage**

The MERN stack is a technology stack for building web applications, consisting of:

1. **MongoDB** (NoSQL database)
2. **Express.js** (Node.js web framework)
3. **React.js** (Front-end JavaScript library)
4. **Node.js** (JavaScript runtime environment)

While knowledge of the MERN stack can be useful for a Data Scientist, it's not a core requirement. If you're interested in developing your skills in the MERN stack, you can take online courses or build personal projects, but your primary focus should be on developing skills in areas relevant to Data Science.


In [221]:
user_input16 = "what is flutter ?"

responses4 = get_response(user_input16,session_id)
print(responses4)

No relevant information found in the vector store.


In [222]:
user_input17 = "what is MERN ?"

responses5 = get_response(user_input17,session_id)
print(responses5)

The MERN stack is a technology stack for building web applications, consisting of:

1. **MongoDB** (NoSQL database)
2. **Express.js** (Node.js web framework)
3. **React.js** (Front-end JavaScript library)
4. **Node.js** (JavaScript runtime environment)

It's not directly related to the skills required for a Data Scientist role, but rather a technology stack for building web applications.
